# Imports libraries

In [1]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import time
sns.set_style('whitegrid')
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import warnings
warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
import xlwt
from xlwt import Workbook
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from scipy.stats.mstats import winsorize
from pylab import rcParams
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import explained_variance_score
!pip install vowpalwabbit

     |████████████████████████████████| 2.6MB 4.2MB/s 


# Import dataset

In [2]:
path = "/content/sample_data/Admission_Predict_kaggle.csv"
df = pd.read_csv(path)

In [3]:
df.shape

(400, 8)

In [4]:
df.columns

Index(['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR ', 'CGPA',
       'Research', 'Chance_of_Admit '],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   GRE Score          400 non-null    int64  
 1   TOEFL Score        400 non-null    int64  
 2   University Rating  400 non-null    int64  
 3   SOP                400 non-null    float64
 4   LOR                400 non-null    float64
 5   CGPA               400 non-null    float64
 6   Research           400 non-null    int64  
 7   Chance_of_Admit    400 non-null    float64
dtypes: float64(4), int64(4)
memory usage: 25.1 KB


# Split the dataset

In [6]:
X = df.drop(['Chance_of_Admit '], axis=1)
y = df['Chance_of_Admit ']

In [7]:
# Workbook is created
wb = Workbook()
sheet1 = wb.add_sheet('Admission_Predict')
sheet1.write(0, 1, 'Dataset name')
sheet1.write(0, 2, 'Algorithm Name')
sheet1.write(0, 3, 'Cross Validation [1-10]')
sheet1.write(0, 4, 'Hyper-Parameters Values')
sheet1.write(0, 5, 'Mean Squared Error')
sheet1.write(0, 6, 'Mean Absolute Error')
sheet1.write(0, 7, 'Median Absolute Error')
sheet1.write(0, 8, 'Max Error')
sheet1.write(0, 9, 'Explained Variance')
sheet1.write(0, 10, 'Train Time')
sheet1.write(0, 11, 'Inference Time')

sheet1.write(1, 1, 'Admission_Predict_Kaggle')

# VWRegressor evaluation

In [8]:
from vowpalwabbit.sklearn_vw import VWRegressor
from sklearn.model_selection import RandomizedSearchCV
# manual nested cross-validation for random forest on a classification dataset
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from scipy.stats import uniform
from scipy.stats import loguniform
from scipy.stats import randint

# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
mean_squared_results = list()
mean_absolute_results = list()
median_absolute_results = list()
max_err_results = list()
explained_var_results = list()
train_time = list()
test_time = list()
params = list()
i = 1
sheet1.write(1, 2, 'VWRegressor')
for train_ix, test_ix in cv_outer.split(X):
	# split data
	X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
	y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
	# configure the cross-validation procedure
	cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
	# define the model
	vw = VWRegressor()
	# define search space
	distributions = {
     'l1': loguniform(1e-8, 1e-1),
     'l2': loguniform(1e-8, 1e-1),
     'l': loguniform(0.01, 10),
     'power_t': uniform(0.01, 1),
     'random_weights': ["on", "off"],
     'loss_function': ["squared", "hinge", "logistic", "quantile", "poisson"],
     'passes': randint(1, 10),
  }
	# define search
	clf = RandomizedSearchCV(vw, distributions, random_state=0, cv=cv_inner, n_iter=50)
	# execute search
	search = clf.fit(X_train, y_train)
	# get the best performing model fit on the whole training set
	best_model = search.best_estimator_
	# print(i)
	# print(best_model.get_params())
	params.append(str(best_model.get_params()))
	start = time.time()
	best_model = best_model.fit(X_train, y_train)
	end = time.time()
	train = end - start
	# evaluate model on the hold out dataset
	start = time.time()
	yhat = best_model.predict(X_test)
	end = time.time()
	test = end - start
	# evaluate the model
	mean_squared = mean_squared_error(y_test, yhat)
	mean_absolute = mean_absolute_error(y_test, yhat)
	median_absolute = median_absolute_error(y_test, yhat)
	max_err = max_error(y_test, yhat)
	explained_var = explained_variance_score(y_test, yhat)
	# store the result
	mean_squared_results.append(mean_squared)
	mean_absolute_results.append(mean_absolute)
	median_absolute_results.append(median_absolute)
	max_err_results.append(max_err)
	explained_var_results.append(explained_var)
	# print('Mean Squared Error: %.3f' % (mean_squared_results))
	# print('Mean Absolute Error: %.3f' % (mean_absolute_results))
	# print('Median Absolute Error: %.3f' % (median_absolute_results))
	# print('Max Error: %.3f' % (median_absolute_results))
	# print('Explained Variance: %.3f' % (explained_var_results))
	# print('Train Time: %.3f' % (train_time))
	# print('Test Time: %.3f' % (test_time))
	sheet1.write(i, 3, i)
	sheet1.write(i, 4, str(best_model.get_params()))
	sheet1.write(i, 5, (mean_squared))
	sheet1.write(i, 6, (mean_absolute))
	sheet1.write(i, 7,  (median_absolute))
	sheet1.write(i, 8,  (max_err))
	sheet1.write(i, 9,  (explained_var))
	sheet1.write(i, 10, (train))
	sheet1.write(i, 11,  (test))
	i += 1

# summarize the estimated performance of the model
print('Mean Squared Error: %.3f (%.3f)' % (mean(mean_squared_results), std(mean_squared_results)))
print('Mean Absolute Error: %.3f (%.3f)' % (mean(mean_absolute_results), std(mean_absolute_results)))
print('Median Absolute Error: %.3f (%.3f)' % (mean(median_absolute_results), std(median_absolute_results)))
print('Max Error: %.3f (%.3f)' % (mean(max_err_results), std(max_err_results)))
print('Explained Variance: %.3f (%.3f)' % (mean(explained_var_results), std(explained_var_results)))
print('Train Time: %.3f (%.3f)' % (mean(train_time), std(train_time)))
print('Test Time: %.3f (%.3f)' % (mean(test_time), std(test_time)))

Mean Squared Error: 2.994 (0.069)
Mean Absolute Error: 1.724 (0.020)
Median Absolute Error: 1.729 (0.024)
Max Error: 1.956 (0.018)
Explained Variance: -0.000 (0.000)
Train Time: nan (nan)
Test Time: nan (nan)


# Suggestion for improvement

In [9]:
from vowpalwabbit.sklearn_vw import VWRegressor
from sklearn.model_selection import RandomizedSearchCV
# manual nested cross-validation for random forest on a classification dataset
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold

# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
mean_squared_results = list()
mean_absolute_results = list()
median_absolute_results = list()
max_err_results = list()
explained_var_results = list()
train_time = list()
test_time = list()
j = 1
sheet1.write(11, 2, 'Suggestion for improvement')
for train_ix, test_ix in cv_outer.split(X):
	# split data
	X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
	y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
	# configure the cross-validation procedure
	cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
	# define the model
	vw_squared = VWRegressor(loss_function='squared')
	vw_hinge = VWRegressor(loss_function='hinge')
	vw_logistic = VWRegressor(loss_function='logistic')
	vw_quantile = VWRegressor(loss_function='quantile')
	vw_poisson = VWRegressor(loss_function='poisson')
	# define search space
	distributions = {
     'l1': loguniform(1e-8, 1e-1),
     'l2': loguniform(1e-8, 1e-1),
     'l': loguniform(0.01, 10),
     'power_t': uniform(0.01, 1),
     'random_weights': ["on", "off"],
     'passes': randint(1, 10),
	}
	# define search
	clf_squared = RandomizedSearchCV(vw_squared, distributions, random_state=0, cv=cv_inner, n_iter=50)
	clf_hinge = RandomizedSearchCV(vw_hinge, distributions, random_state=0, cv=cv_inner, n_iter=50)
	clf_logistic = RandomizedSearchCV(vw_logistic, distributions, random_state=0, cv=cv_inner, n_iter=50)
	clf_quantile = RandomizedSearchCV(vw_quantile, distributions, random_state=0, cv=cv_inner, n_iter=50)
	clf_poisson = RandomizedSearchCV(vw_poisson, distributions, random_state=0, cv=cv_inner, n_iter=50)
	# execute search
	search_squared = clf_squared.fit(X_train, y_train)
	search_hinge = clf_hinge.fit(X_train, y_train)
	search_logistic = clf_logistic.fit(X_train, y_train)
	search_quantile = clf_quantile.fit(X_train, y_train)
	search_poisson = clf_poisson.fit(X_train, y_train)
	# get the best performing model fit on the whole training set
	best_model_squared = search_squared.best_estimator_
	best_model_hinge = search_hinge.best_estimator_
	best_model_logistic = search_logistic.best_estimator_
	best_model_quantile = search_quantile.best_estimator_
	best_model_poisson = search_poisson.best_estimator_
	# print(j)
	# print(best_model_squared.get_params())
	# print(best_model_hinge.get_params())
	# print(best_model_logistic.get_params())
	# print(best_model_quantile.get_params())
	# print(best_model_poisson.get_params())
	# fit the model
	start = time.time()
	best_model_squared = best_model_squared.fit(X_train, y_train)
	best_model_hinge = best_model_hinge.fit(X_train, y_train)
	best_model_logistic = best_model_logistic.fit(X_train, y_train)
	best_model_quantile = best_model_quantile.fit(X_train, y_train)
	best_model_poisson = best_model_poisson.fit(X_train, y_train)
	end = time.time()
	train = end - start
	# evaluate model on the hold out dataset
	start = time.time()
	yhat_squared = best_model_squared.predict(X_test)
	yhat_hinge = best_model_hinge.predict(X_test)
	yhat_logistic = best_model_logistic.predict(X_test)
	yhat_quantile = best_model_quantile.predict(X_test)
	yhat_poisson = best_model_poisson.predict(X_test)
	avg = []
	for i in range(len(yhat_squared)):
		avg.append((yhat_squared[i] + yhat_hinge[i] + yhat_logistic[i] + yhat_quantile[i] + yhat_poisson[i]) / 5)
	end = time.time()
	test = end - start
	# evaluate the model
	mean_squared = mean_squared_error(y_test, avg)
	mean_absolute = mean_absolute_error(y_test, avg)
	median_absolute = median_absolute_error(y_test, avg)
	max_err = max_error(y_test, avg)
	explained_var = explained_variance_score(y_test, avg)
	# store the result
	mean_squared_results.append(mean_squared)
	mean_absolute_results.append(mean_absolute)
	median_absolute_results.append(median_absolute)
	max_err_results.append(max_err)
	explained_var_results.append(explained_var)
	sheet1.write(j + 10, 3, j)
	sheet1.write(j + 10, 4, str(best_model_squared.get_params()))
	sheet1.write(j + 10, 5, (mean_squared))
	sheet1.write(j + 10, 6,  (mean_absolute))
	sheet1.write(j + 10, 7, (median_absolute))
	sheet1.write(j + 10, 8, (max_err))
	sheet1.write(j + 10, 9, (explained_var))
	sheet1.write(j + 10, 10, (train))
	sheet1.write(j + 10, 11, (test))
	j += 1

# summarize the estimated performance of the model
print('Mean Squared Error: %.3f (%.3f)' % (mean(mean_squared_results), std(mean_squared_results)))
print('Mean Absolute Error: %.3f (%.3f)' % (mean(mean_absolute_results), std(mean_absolute_results)))
print('Median Absolute Error: %.3f (%.3f)' % (mean(median_absolute_results), std(median_absolute_results)))
print('Max Error: %.3f (%.3f)' % (mean(max_err_results), std(max_err_results)))
print('Explained Variance: %.3f (%.3f)' % (mean(explained_var_results), std(explained_var_results)))
print('Train Time: %.3f (%.3f)' % (mean(train_time), std(train_time)))
print('Test Time: %.3f (%.3f)' % (mean(test_time), std(test_time)))

Mean Squared Error: 454.748 (0.867)
Mean Absolute Error: 21.324 (0.020)
Median Absolute Error: 21.329 (0.024)
Max Error: 21.556 (0.018)
Explained Variance: -0.000 (0.000)
Train Time: nan (nan)
Test Time: nan (nan)


# Random forest

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
# manual nested cross-validation for random forest on a classification dataset
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold

# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
mean_squared_results = list()
mean_absolute_results = list()
median_absolute_results = list()
max_err_results = list()
explained_var_results = list()
train_time = list()
test_time = list()
i = 1
sheet1.write(21, 2, 'Random forest')
for train_ix, test_ix in cv_outer.split(X):
	# split data
	X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
	y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
	# configure the cross-validation procedure
	cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
	# define the model
	rf = RandomForestRegressor()
	# define search space
	distributions = {
     'criterion': ['mse', 'mae'],
     'min_samples_split': [2, 3, 4, 5],
     'min_samples_leaf': [1, 2, 3, 4],
     'min_weight_fraction_leaf': uniform(0.01, 0.5),
     'max_features': ['auto', 'sqrt', 'log2']
  }
	# define search
	clf = RandomizedSearchCV(rf, distributions, random_state=0, cv=cv_inner, n_iter=50)
	# execute search
	search = clf.fit(X_train, y_train)
	# get the best performing model fit on the whole training set
	best_model = search.best_estimator_
	# print(i)
	# print(best_model.get_params())
	start = time.time()
	best_model = best_model.fit(X_train, y_train)
	end = time.time()
	train = end - start
	# evaluate model on the hold out dataset
	start = time.time()
	yhat = best_model.predict(X_test)
	end = time.time()
	test = end - start
	# evaluate the model
	mean_squared = mean_squared_error(y_test, yhat)
	mean_absolute = mean_absolute_error(y_test, yhat)
	median_absolute = median_absolute_error(y_test, yhat)
	max_err = max_error(y_test, yhat)
	explained_var = explained_variance_score(y_test, yhat)
	# store the result
	mean_squared_results.append(mean_squared)
	mean_absolute_results.append(mean_absolute)
	median_absolute_results.append(median_absolute)
	max_err_results.append(max_err)
	explained_var_results.append(explained_var)
	sheet1.write(i + 20, 3, i)
	sheet1.write(i + 20, 4, str(best_model_squared.get_params()))
	sheet1.write(i + 20, 5, (mean_squared))
	sheet1.write(i + 20, 6, (mean_absolute))
	sheet1.write(i + 20, 7, (median_absolute))
	sheet1.write(i + 20, 8, (max_err))
	sheet1.write(i + 20, 9, (explained_var))
	sheet1.write(i + 20, 10, (train))
	sheet1.write(i + 20, 11, (test))
	i += 1

wb.save('Admission_Predict.xls')
# summarize the estimated performance of the model
print('Mean Squared Error: %.3f (%.3f)' % (mean(mean_squared_results), std(mean_squared_results)))
print('Mean Absolute Error: %.3f (%.3f)' % (mean(mean_absolute_results), std(mean_absolute_results)))
print('Median Absolute Error: %.3f (%.3f)' % (mean(median_absolute_results), std(median_absolute_results)))
print('Max Error: %.3f (%.3f)' % (mean(max_err_results), std(max_err_results)))
print('Explained Variance: %.3f (%.3f)' % (mean(explained_var_results), std(explained_var_results)))
print('Train Time: %.3f (%.3f)' % (mean(train_time), std(train_time)))
print('Test Time: %.3f (%.3f)' % (mean(test_time), std(test_time)))

Mean Squared Error: 0.005 (0.001)
Mean Absolute Error: 0.048 (0.006)
Median Absolute Error: 0.034 (0.005)
Max Error: 0.210 (0.042)
Explained Variance: 0.781 (0.053)
Train Time: nan (nan)
Test Time: nan (nan)
